In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")
!ls

 Complete.pdf				  PDF_Folder_Biomedical
'covid_question_answer_dataset (1).csv'   PDF_Folder_Financial
 covid_question_answer_dataset.csv	  PDF_Folder_Scientific
 CrossDomainAnalysis.ipynb		 ' QA_system_for_BioMedical_data.ipynb'
 DistilBERT.ipynb			  QA_system_for_scientific_data.ipynb
 EvaluationScript_financial.ipynb	  qa-with-squad-2.0-dataset-and-bert-base.ipynb
 FinBERT.ipynb				  question_answer_dataset_financial.csv
'FineTuningDistillBert_SQAUD .ipynb'	  scientific_question_answer_dataset.csv
 FineTuningDistillBert_SQAUD_.ipynb	  Untitled0.ipynb
 Make_dataset_from_PDFs.ipynb		  Untitled1.ipynb
 my_awesome_qa_model


In [ ]:
!pip install pdfplumber          # Python library for extracting information from PDF
!pip install PyPDF2              #library for reading and manipulating PDF files in Python.
!pip install nltk                #library provides tools for tasks such as tokenization, stemming, lemmatization
!pip install -U gensim           # library for topic modeling and document similarity analysis
!pip install accelerate==0.20.3  #library for GPU acceleration in Python
!pip install transformers -U     #library for working with pre-trained models in transformer-based models

import pdfplumber
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from transformers import pipeline
import torch
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
from sklearn.metrics import f1_score
from sklearn.metrics import f1_score
import re
import pandas as pd
import nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 843.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 50.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


Read the Amazon's annual report in PDF format with pdfplumber to extract  test for analysis

In [ ]:
pdf = pdfplumber.open("/content/gdrive/MyDrive/Colab Notebooks/PDF_Folder_Financial/Q1-2023-Amazon-Earnings-Release.pdf")
print(len(pdf.pages))
pdf_txt= ""
for i in range(len(pdf.pages)):
    pdf_txt +=pdf.pages[i].extract_text()
pdf.close()

15


In [ ]:
pdf_txt

'AMAZON.COM ANNOUNCES FIRST QUARTER RESULTS\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results\nfor its first quarter ended March 31, 2023.\n• Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in first quarter 2022.\nExcluding the $2.4 billion unfavorable impact from year-over-year changes in foreign exchange rates throughout the\nquarter, net sales increased 11% compared with first quarter 2022.\n• North America segment sales increased 11% year-over-year to $76.9 billion.\n• International segment sales increased 1% year-over-year to $29.1 billion, or increased 9% excluding changes\nin foreign exchange rates.\n• AWS segment sales increased 16% year-over-year to $21.4 billion.\n• Operating income increased to $4.8 billion in the first quarter, compared with $3.7 billion in first quarter 2022. First\nquarter 2023 operating income includes approximately $0.5 billion of charges related to estim

 Tokenizes the sentences using the sent_tokenize function from NLTK

In [ ]:
pdf_txt = """ AMAZON.COM ANNOUNCES THIRD QUARTER RESULTS
SEATTLE—(BUSINESS WIRE) October 26, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial
results for its third quarter ended September 30, 2023.
• Net sales increased 13% to $143.1 billion in the third quarter, compared with $127.1 billion in third quarter 2022.
Excluding the $1.4 billion favorable impact from year-over-year changes in foreign exchange rates throughout the
quarter, net sales increased 11% compared with third quarter 2022.
• North America segment sales increased 11% year-over-year to $87.9 billion.
• International segment sales increased 16% year-over-year to $32.1 billion, or increased 11% excluding
changes in foreign exchange rates.
• AWS segment sales increased 12% year-over-year to $23.1 billion.
• Operating income increased to $11.2 billion in the third quarter, compared with $2.5 billion in third quarter 2022.
• North America segment operating income was $4.3 billion, compared with an operating loss of $0.4 billion in
third quarter 2022.
• International segment operating loss was $0.1 billion, compared with an operating loss of $2.5 billion in third
quarter 2022.
• AWS segment operating income was $7.0 billion, compared with operating income of $5.4 billion in third
quarter 2022.
• Net income increased to $9.9 billion in the third quarter, or $0.94 per diluted share, compared with $2.9 billion, or
$0.28 per diluted share, in third quarter 2022.
• Third quarter 2023 net income includes a pre-tax valuation gain of $1.2 billion included in non-operating
income (expense) from the common stock investment in Rivian Automotive, Inc., compared to a pre-tax
valuation gain of $1.1 billion from the investment in third quarter 2022.
• Operating cash flow increased 81% to $71.7 billion for the trailing twelve months, compared with $39.7 billion for
the trailing twelve months ended September 30, 2022.
• Free cash flow improved to an inflow of $21.4 billion for the trailing twelve months, compared with an outflow of
$19.7 billion for the trailing twelve months ended September 30, 2022.
• Free cash flow less principal repayments of finance leases and financing obligations improved to an inflow of
$15.9 billion for the trailing twelve months, compared with an outflow of $28.5 billion for the trailing twelve months
ended September 30, 2022.
• Free cash flow less equipment finance leases and principal repayments of all other finance leases and financing
obligations improved to an inflow of $20.2 billion for the trailing twelve months, compared with an outflow of $21.5
billion for the trailing twelve months ended September 30, 2022.
“We had a strong third quarter as our cost to serve and speed of delivery in our Stores business took another step forward, our
AWS growth continued to stabilize, our Advertising revenue grew robustly, and overall operating income and free cash flow
rose significantly,” said Andy Jassy, Amazon CEO. “The benefits of moving from a single national fulfillment network in the
U.S. to eight distinct regions are exceeding our optimistic expectations, and perhaps most importantly, putting us on pace to
deliver the fastest delivery speeds for Prime customers in our 29-year history. The AWS team continues to innovate and deliver
at a rapid clip, particularly in generative AI, where the combination of our custom AI chips, Amazon Bedrock being the easiest
and most flexible way to build and deploy generative AI applications, and our coding companion (CodeWhisperer) allowing
enterprises to have the equivalent of an experienced engineer who understands all of their proprietary code is driving
momentum with customers, including adidas, Booking.com, GoDaddy, LexisNexis, Merck, Royal Philips, and United Airlines,
all of whom are starting to run generative AI workloads on AWS. Between AWS re:Invent and our 29th holiday shopping
season, this is a particularly action-packed time of year at Amazon and we’re excited for what’s to come.”
Highlights
Obsessing over the customer experience
Amazon obsesses over how to make customers’ lives better and easier every day with new and improved products and services.
This is true for consumers, sellers, brands, developers, enterprises, and creators. For example, Amazon:
• Sold hundreds of millions of items worldwide during Prime Big Deal Days, Amazon’s largest two-day October
holiday kick-off event ever. Prime members across 19 countries saved more than $1 billion on millions of deals. On
the first day of the event, U.S. Prime members purchased more than 25 million items with Same-Day or Next-Day
Delivery, with hundreds of thousands of items delivered within four hours of purchase.
• Announced new Buy with Prime features to help merchants grow their businesses on their own websites, such as the
ability to integrate Buy with Prime with Shopify. The Buy with Prime app for Shopify will make it easier for Shopify
merchants to manage their businesses, with inventory, pricing, and promotions automatically synced in one place. In
addition, Buy with Prime Assist enables merchants to offer 24/7 post-order customer service support at no additional
cost, and the Buy with Prime cart feature allows customers to purchase multiple Buy with Prime products in one
transaction from a single merchant.
• Announced new offerings to help customers get and stay healthy, including an expanded Amazon Clinic virtual health
care marketplace now available across the U.S. to provide care for more than 35 conditions, and 60-minute delivery of
medications through Amazon Pharmacy using Prime Air drones in College Station, Texas. Additionally, Amazon
Pharmacy partnered with Blue Shield of California on a first-of-its-kind model to provide more affordable pharmacy
care to its members, where Amazon Pharmacy will offer fast, free home delivery of prescription medications to more
than 4.8 million Blue Shield of California members, starting in 2025.
• Partnered with BuzzFeed, Hearst Newspapers, Pinterest, Raptive, and Ziff Davis brands like Lifehacker and Mashable,
among others, to display sponsored product ads on their apps and websites for products sold in Amazon’s U.S. store.
These third-party ad partnerships will make it easier for customers to move from inspiration to buying in one or two
clicks.
• Attracted 15.1 million viewers for the Thursday Night Football (TNF) season opener, Prime Video’s most watched
TNF game ever, according to Nielsen. Through the first six games, TNF averaged 12.9 million viewers, an increase of
25% from last season’s six-game average, according to Nielsen. Season to date, the median age for TNF viewers is 47,
which is seven years younger than the median age watching the NFL on linear networks.
• Debuted the second season of Amazon Original series The Summer I Turned Pretty, which had double the global
audience in the first three days compared to Season One. In addition, for the first seven weeks after its debut, it was the
most viewed series on Prime Video in over 100 countries.
• Released Amazon Original films and series A Million Miles Away, the real-life story of NASA flight engineer José
Hernández; Red, White & Royal Blue, the romantic comedy based on Casey McQuiston’s New York Times best-selling
novel; The Burial, a true story exposing corporate corruption and racial injustice, starring Jamie Foxx and Tommy Lee
Jones; Sitting in Bars with Cake, the story of baking and best friends; and Gen V, an expansion of The Boys Universe.
Amazon also premiered the second seasons of the hit young-adult drama The Summer I Turned Pretty and the fantasy
series The Wheel of Time, and rebooted the globally beloved soap opera Neighbours.
• Premiered new films from Amazon MGM Studios, including American Fiction, which won the People’s Choice
Award at the Toronto International Film Festival. The film will begin its theatrical release in December. Amazon
MGM Studios also released the indie film Bottoms, which opened in select U.S. markets on August 25.
• Launched Prime Video Channels and the Prime Video Store in Chile and Colombia, and grew Prime Video’s
international slate of content with more than 40 local Amazon Originals and live sporting events. This includes: The
Lost Flowers of Alice Hart (Australia), Cangaço Novo (Brazil), Los Iniciados (Colombia), All or Nothing: The
German National Team in Qatar (Germany), Made in Heaven (India), The Ferragnez - The Series (Italy), Sanremo
Special (Italy), and Wilderness (UK).
• Entered a strategic collaboration to advance generative AI with Anthropic. As part of the expanded collaboration,
Anthropic selected AWS as its primary cloud provider and will build and deploy future generations of its foundation
models on AWS Trainium and AWS Inferentia chips. Anthropic plans to run the majority of its workloads on AWS
and will also provide AWS customers with access to future generations of its foundation models via Amazon Bedrock.
Amazon developers and engineers will have the opportunity to build with Anthropic models to enhance existing
applications and create new customer experiences.
• Announced new AI-driven commitments and expansions from AWS customers.
• BMW Group selected AWS to power its next-generation of automated driving platform, which will use
BMW's preexisting Cloud Data Hub on AWS, and will use AWS compute, generative AI, Internet of Things
(IoT), machine learning (ML), and storage capabilities to help accelerate the delivery of highly automated
BMW vehicles.
• UK-based bank NatWest expanded its partnership with AWS to accelerate the responsible use of AI and cocreate AI products on foundation models available through Amazon Bedrock to enhance the financial wellbeing of its 10 million customers.
• Occidental, an international energy company and carbon management leader, selected AWS as its preferred
cloud provider to improve operational efficiencies, reduce IT infrastructure costs, and develop systems for
carbon removal plants, which are expected to capture 1 million metric tons per year. Occidental is leveraging
AWS analytics and ML services like Amazon Elastic Compute Cloud (Amazon EC2), Amazon Elastic Block
Store (Amazon EBS) to scale IT resources, and AWS Lake Formation to increase efficiencies and extract
greater value from operational data.
• PwC announced a collaboration with AWS to deliver a goal of $800 million in business value to clients and
strengthen PwC’s offerings, including cloud migration and modernization, data and analytics, cybersecurity
and risk, customer experience, managed services, generative AI, and climate reporting and transition to netzero carbon emissions.
• EVERSANA, provider of commercial services to the global life sciences industry, teamed up with AWS to
use Amazon Bedrock to accelerate generative AI use cases to help pharmaceutical and life science
manufacturers drive efficiencies and business value while improving patient outcomes.
• DS Smith, a sustainable packaging provider, will leverage AWS’s global infrastructure, advanced analytics,
IoT, AI, and ML to derive deeper business intelligence, deliver new digital customer experiences, and drive
automation at scale across its supply chain, including real-time monitoring of manufacturing capacity and
supply chain management to support sustainability targets.
• Abdul Latif Jameel, an internationally diversified business, selected AWS as its preferred cloud provider to
use AWS analytics, ML, and Amazon Bedrock to develop generative AI applications that will help car
manufacturers offer digital showroom experiences for customers and enhance in-car experiences, while
reducing development costs.
• South Korean game developer NCSOFT launched seven large language models (LLMs) on Amazon
SageMaker JumpStart to democratize generative AI in Korea and accelerate development of new gaming and
creative applications.
• Genpact, a global professional services firm, expanded its relationship with AWS to transform its financial
crime risk operations by leveraging generative AI and LLMs. Genpact is accelerating efficiencies and impact
for their clients by integrating their proprietary cloud-based financial crime suite with Amazon Bedrock.
• Continued to expand AWS’s infrastructure footprint to support customers by launching the AWS Israel (Tel Aviv)
Region and a new AWS Local Zone in Phoenix, Arizona. The AWS Israel (Tel Aviv) Region is estimated to support
an average of 7,700 full-time equivalent jobs annually through a planned investment of $7.2 billion through 2037.  """

import nltk
nltk.download('punkt')
tokens = nltk.sent_tokenize(pdf_txt)
for t in tokens:
    print(t, "\n")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


 AMAZON.COM ANNOUNCES THIRD QUARTER RESULTS
SEATTLE—(BUSINESS WIRE) October 26, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial
results for its third quarter ended September 30, 2023. 

• Net sales increased 13% to $143.1 billion in the third quarter, compared with $127.1 billion in third quarter 2022. 

Excluding the $1.4 billion favorable impact from year-over-year changes in foreign exchange rates throughout the
quarter, net sales increased 11% compared with third quarter 2022. 

• North America segment sales increased 11% year-over-year to $87.9 billion. 

• International segment sales increased 16% year-over-year to $32.1 billion, or increased 11% excluding
changes in foreign exchange rates. 

• AWS segment sales increased 12% year-over-year to $23.1 billion. 

• Operating income increased to $11.2 billion in the third quarter, compared with $2.5 billion in third quarter 2022. 

• North America segment operating income was $4.3 billion, compared with an operating lo

functions for cleaning sentences using the gensim library and regular expressions

In [ ]:
def clean_sentence(sentence, stopwords=False):
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
  if stopwords:
    sentence = remove_stopwords(sentence)
  return sentence

def get_cleaned_sentences(tokens, stopwords=False):
  cleaned_sentences = []
  for row in tokens:
    cleaned = clean_sentence(row, stopwords)
    cleaned_sentences.append(cleaned)
  return cleaned_sentences


In [ ]:
cleaned_sentences = get_cleaned_sentences(tokens, stopwords=True)
cleaned_sentences_with_stopwords = get_cleaned_sentences(tokens, stopwords=False)
print(cleaned_sentences)
print(cleaned_sentences_with_stopwords)

['amazoncom announces quarter results seattlebusiness wire october 26 2023amazoncom nasdaq amzn today announced financial results quarter ended september 30 2023', 'net sales increased 13 1431 billion quarter compared 1271 billion quarter 2022', 'excluding 14 billion favorable impact yearoveryear changes foreign exchange rates quarter net sales increased 11 compared quarter 2022', 'north america segment sales increased 11 yearoveryear 879 billion', 'international segment sales increased 16 yearoveryear 321 billion increased 11 excluding changes foreign exchange rates', 'aws segment sales increased 12 yearoveryear 231 billion', 'operating income increased 112 billion quarter compared 25 billion quarter 2022', 'north america segment operating income 43 billion compared operating loss 04 billion quarter 2022', 'international segment operating loss 01 billion compared operating loss 25 billion quarter 2022', 'aws segment operating income 70 billion compared operating income 54 billion quar

# Implenting with bert-large and bert-base models with QA library

In [ ]:
# Experiment with different models
qa_pipeline_large = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")
qa_pipeline_base = pipeline("question-answering", model="bert-base-uncased")

# Assign cleaned sentences to context
context = " ".join(cleaned_sentences)
question = "What is the topic of the document?"

# Get answers from different models
answer_large = qa_pipeline_large(question=question, context=context)
answer_base = qa_pipeline_base(question=question, context=context)

# Print answers and scores
print("Large Model Answer:", answer_large["answer"])
print("Large Model Score:", answer_large["score"])

print("Base Model Answer:", answer_base["answer"])
print("Base Model Score:", answer_base["score"])

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Large Model Answer: corporate corruption racial injustice
Large Model Score: 0.4413483440876007
Base Model Answer: supply chain management support sustainability targets abdul
Base Model Score: 3.550049223122187e-05


Question Answering with a Fine-Tuned BERT

Bidirectional Encoder Representations from Transformers

Instead of looking at words in isolation, BERT, a transformer based model attempts to use the context of words to get embeddings. BERT uses several concepts of deep learning come up with a model that looks at context in a bidirectional fashion, leveraging information from the entire sentences as a whole through self-attention.

Example: Suppose Question is "How I can cross a road when their is no zebra crossing." and this question very similar to query "How I can cross road on zebra crossing." and thier is a chance that above two method give wrong answers but could handle that question. And we don't remove stoping words in the document because if it remove word like in above example "no" meaning of whole sentance become change. Bert uses stoping words to make relations between words.

What is Model Fine-Tuning?

BERT (Bidirectional Encoder Representations from Transformers) is a big neural network architecture, with a huge number of parameters, that can range from 100 million to over 300 million. So, training a BERT model from scratch on a small dataset would result in overfitting.

So, it is better to use a fine-tuned pre-trained BERT model that was trained on a huge dataset, as a starting point.we will use BERT for Question answering trained on SQAUD dataset.

# Load Fine-Tuned BERT-large

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Load the tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

 lets implement and test the model on small text as BERT won't take the longer documents because of the limitation of 512 tokens.

In [ ]:
answer_text = """The ability of the machine to infer knowledge from
the user documents can be tested based on its ability to answer
the question asked. Conventional Artificial Neural Network
(ANN) models for knowledge extraction only answer to the
questions which are simple and objective as they don't analyze
the questions and don't try to understand what really the content
of document mean. The proposed question answering system
(QAS) uses deep cases along with ANN to understand the
contents of the documents. We divide the sentences of natural
language into knowledge units and assign deep case to each word
to improve the quality of knowledge extraction."""

question = "How to improve the quality of knowledge extraction?"

In [ ]:
answer_text22 = """Net sales increased 13% to $143.1 billion in the third quarter, compared with $127.1 billion in third quarter 2022.
Excluding the $1.4 billion favorable impact from year-over-year changes in foreign exchange rates throughout the
quarter, net sales increased 11% compared with third quarter 2022.
"""

question22 = "How much was the net sales in the year 2022?"

Apply the tokenizer to the input text, treating them as a text-pair.

In [ ]:
input_ids = tokenizer.encode(question22, answer_text22, max_length=512, truncation=True)
print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 80 tokens.


Just to see exactly what the tokenizer is doing, let's print out the tokens with their IDs.

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    if id == tokenizer.sep_token_id:
        print('')
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')

[CLS]           101
how           2,129
much          2,172
was           2,001
the           1,996
net           5,658
sales         4,341
in            1,999
the           1,996
year          2,095
202          16,798
##2           2,475
?             1,029

[SEP]           102

net           5,658
sales         4,341
increased     3,445
13            2,410
%             1,003
to            2,000
$             1,002
143          16,065
.             1,012
1             1,015
billion       4,551
in            1,999
the           1,996
third         2,353
quarter       4,284
,             1,010
compared      4,102
with          2,007
$             1,002
127          13,029
.             1,012
1             1,015
billion       4,551
in            1,999
third         2,353
quarter       4,284
202          16,798
##2           2,475
.             1,012
excluding    13,343
the           1,996
$             1,002
1             1,015
.             1,012
4             1,018
billion       4,55

segment embeddings to enter question and answer text into transformer

In [ ]:
sep_index = input_ids.index(tokenizer.sep_token_id)
num_seg_a = sep_index + 1

num_seg_b = len(input_ids) - num_seg_a
#Here We Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b
# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

Here we feed answer_text and question into model


In [ ]:
outputs = model(input_ids=torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

Here we find the highest 'start' and 'end' scores and the combine the tokens in the answer and print the answer

In [ ]:
start_index = torch.argmax(outputs.start_logits)
end_index = torch.argmax(outputs.end_logits)

In [ ]:
# Print the keys of the output dictionary to inspect the structure
print(outputs.keys())

odict_keys(['start_logits', 'end_logits'])


In [ ]:
answer = tokens[start_index]
#answer = tokens[end_index]

# Select the remaining answer tokens and join them with whitespace.
for i in range(start_index + 1, end_index + 1):
  # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
        # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]
print('Answer: "' + answer + '"')


Answer: "$ 127 . 1 billion"


lets make function for QA of overall code

In [ ]:
def answer_question(question, answer_text):
    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
    print('Query has {:,} tokens.\n'.format(len(input_ids)))
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    outputs = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    score = float(torch.max(start_index))
    answer_start = torch.argmax(start_index)
    answer_end = torch.argmax(end_index)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[start_index]
    for i in range(start_index + 1, end_index + 1):

        if tokens[i][0:2] == ' ':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]
    return answer, score

Lets test the model on random samples of question answer pairs.

In [ ]:
answer_text = """The ability of the machine to infer knowledge from
the user documents can be tested based on its ability to answer
the question asked. Conventional Artificial Neural Network
(ANN) models for knowledge extraction only answer to the
questions which are simple and objective as they don't analyze
the questions and don't try to understand what really the content
of document mean. The proposed question answering system
(QAS) uses deep cases along with ANN to understand the
contents of the documents. We divide the sentences of natural
language into knowledge units and assign deep case to each word
to improve the quality of knowledge extraction."""

question = "How to improve the quality of knowledge extraction?"
ground_truth = "divide the sentences of natural language into knowledge units and assign deep case to each word"

question1 = "What is the capital of France?"
answer_text1 = "The capital of France is Paris."
ground_truth1 = "France"

question2 = "What is Artificial Intelligence?"
answer_text2 = "AI, or Artificial Intelligence, refers to the simulation of human intelligence in machines that are programmed to think and learn like humans."
ground_truth2 = "simulation of human intelligence"

In [ ]:
answer_question(question1,answer_text1)

Query has 17 tokens.



('paris', 14.0)

In [ ]:
answer_question(question2,answer_text2)

Query has 33 tokens.



('the simulation of human intelligence', 15.0)

In [ ]:
answer_question(question,answer_text)

Query has 130 tokens.



('divide the sentences of natural language into knowledge units and assign deep case to each word',
 105.0)

# Modify function to calculate F1-score

To calculate the F1 score, it need to compare the predicted answer with the ground truth answer.

In [ ]:
def answer_question_F1_Score(question, answer_text, ground_truth):
    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
    print('The input has a total of {:} tokens.'.format(len(input_ids)))
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    # Construct the list of 0s and 1s.
    segment_ids = [0] * num_seg_a + [1] * num_seg_b
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)
    outputs = model(input_ids=torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    predicted_answer = tokens[start_index]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_index + 1, end_index + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            predicted_answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            predicted_answer += ' ' + tokens[i]

    print('Predicted Answer: "' + predicted_answer + '"')
    print('Ground Truth Answer: "' + ground_truth + '"')
    # Calculate F1 score
    y_true = [0] * len(tokens)  # 0 for tokens outside the answer
    y_true[start_index:end_index + 1] = [1] * (end_index - start_index + 1)  # 1 for tokens within the answer
    y_pred = [1 if i >= start_index and i <= end_index else 0 for i in range(len(tokens))]
    f1 = f1_score(y_true, y_pred)
    print('F1 Score:', f1)

Here lets call the function and pass question as 1st parameter and content as 2nd parameter and we get the final answer

In [ ]:
answer_question_F1_Score(question2, answer_text2, ground_truth2)


The input has a total of 33 tokens.
Predicted Answer: "the simulation of human intelligence"
Ground Truth Answer: "simulation of human intelligence"
F1 Score: 1.0


Limitation of Bert that it process only upto 512 tokens So we can implement new method to make it for whole document.

In [ ]:
pdf_txt

" AMAZON.COM ANNOUNCES THIRD QUARTER RESULTS\nSEATTLE—(BUSINESS WIRE) October 26, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial\nresults for its third quarter ended September 30, 2023.\n• Net sales increased 13% to $143.1 billion in the third quarter, compared with $127.1 billion in third quarter 2022.\nExcluding the $1.4 billion favorable impact from year-over-year changes in foreign exchange rates throughout the\nquarter, net sales increased 11% compared with third quarter 2022.\n• North America segment sales increased 11% year-over-year to $87.9 billion.\n• International segment sales increased 16% year-over-year to $32.1 billion, or increased 11% excluding\nchanges in foreign exchange rates.\n• AWS segment sales increased 12% year-over-year to $23.1 billion.\n• Operating income increased to $11.2 billion in the third quarter, compared with $2.5 billion in third quarter 2022.\n• North America segment operating income was $4.3 billion, compared with an operating loss

In [ ]:
user_question1 = "How aws helped amazon to grow in the year 2022?"

We need to split the sentences according to tokens i.e, we can split upto 512 tokens after that we need to add some special token like [SEP], [CLS].

Tokenization: Tokenization is the process of converting a string of text into a list of tokens (individual words/punctuation) and/or token IDs (integers that map a word to a vector representation of that word in an embedding array).

Here we are using the tokenizers encode_plus method to create our tokens from the text string
add_special_tokens=True adds special BERT tokens like [CLS], [SEP], and [PAD] to our new ‘tokenized’ encodings
max_length=512 tells the encoder the target length of our encodings
truncation=True ensures we cut any sequences that are longer than the specified max_length .
padding="max_length" tells the encoder to pad any sequences that are shorter than the max_length with padding tokens.

In [ ]:
tokens = tokenizer.encode_plus(user_question1, pdf_txt, add_special_tokens=True, max_length=512, truncation=True, padding="max_length")
tokens

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'input_ids': [101, 2129, 22091, 2015, 3271, 9733, 2000, 4982, 1999, 1996, 2095, 16798, 2475, 1029, 102, 9733, 1012, 4012, 17472, 2353, 4284, 3463, 5862, 1517, 1006, 2449, 7318, 1007, 2255, 2656, 1010, 16798, 2509, 1517, 9733, 1012, 4012, 1010, 4297, 1012, 1006, 17235, 2850, 4160, 1024, 2572, 2480, 2078, 1007, 2651, 2623, 3361, 3463, 2005, 2049, 2353, 4284, 3092, 2244, 2382, 1010, 16798, 2509, 1012, 1528, 5658, 4341, 3445, 2410, 1003, 2000, 1002, 16065, 1012, 1015, 4551, 1999, 1996, 2353, 4284, 1010, 4102, 2007, 1002, 13029, 1012, 1015, 4551, 1999, 2353, 4284, 16798, 2475, 1012, 13343, 1996, 1002, 1015, 1012, 1018, 4551, 11119, 4254, 2013, 2095, 1011, 2058, 1011, 2095, 3431, 1999, 3097, 3863, 6165, 2802, 1996, 4284, 1010, 5658, 4341, 3445, 2340, 1003, 4102, 2007, 2353, 4284, 16798, 2475, 1012, 1528, 2167, 2637, 6903, 4341, 3445, 2340, 1003, 2095, 1011, 2058, 1011, 2095, 2000, 1002, 6584, 1012, 1023, 4551, 1012, 1528, 2248, 6903, 4341, 3445, 2385, 1003, 2095, 1011, 2058, 1011, 2095, 200

It return a dictionary containing three key-value pairs, input_ids , token_type_ids , and attention_mask . We have also added return_tensors='pt' to return PyTorch tensors from the tokenizer (rather than Python lists).

In [ ]:
tokens = tokenizer.encode_plus(pdf_txt, add_special_tokens=False, return_tensors='pt')
tokens

Token indices sequence length is longer than the specified maximum sequence length for this model (2612 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[ 9733,  1012,  4012,  ..., 18540,  2581,  1012]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [ ]:
input_id_chunks = tokens['input_ids'][0].split(510)
mask_chunks = tokens['attention_mask'][0].split(510)

for tensor in input_id_chunks:
  print(len(tensor))

510
510
510
510
510
62


In [ ]:
a = torch.arange(10)
a

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
torch.cat(
    [torch.Tensor([101]), a, torch.Tensor([102])]
)

tensor([101.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9., 102.])

Preparing The Chunks

Now we have our tokenized tensor; we need to break it into chunks of no more than 510 tokens. We choose 510 rather than 512 to leave two places spare to add our [CLS] and [SEP] tokens.

Split

We apply the split method to both our input IDs and attention mask tensors (we don’t need the token type IDs and can discard them). Now we have three chunks for each tensor set. Note that we will need to add padding to the final chunk as it will not satisfy the tensor size of 512 required by BERT.

CLS and SEP

Next, we add the start of sequence [CLS] and separator [SEP] tokens. For this, we can use the torch.cat function, which concatenates a list of tensors. Our tokens are already in token ID format, so we can refer to the special tokens table above to create the token ID versions of our [CLS] and [SEP] tokens. Because we are doing this for multiple tensors, we place the torch.cat function into a for-loop and perform the concatenation for each of our chunks individually. Additionally, our attention mask chunks are concatenated with 1s rather than 101 and 102. We do this because the attention mask does not contain token IDs but instead a set of 1s and 0s. Zeros in the attention mask represent the location of padding tokens (which we will add next), and as [CLS] and [SEP] are not padding tokens, they are represented with 1s.

Padding

We need to add padding to our tensor chunks to ensure they satisfy the 512 tensorlength required by BERT. Our first two chunks don’t require any padding as they already satisfy this length requirement, but the final chunks do. To check if a chunk requires padding, we add an if-statement that checks the tensor length. If the tensor is shorter than 512 tokens, we add padding using the torch.cat function. We should add this statement to the same for-loop where we add our [CLS] and [SEP] tokens

In [ ]:
chunksize = 512

input_id_chunks = list(tokens['input_ids'][0].split(chunksize - 2))
mask_chunks = list(tokens['attention_mask'][0].split(chunksize - 2))

for i in range(len(input_id_chunks)):
    input_id_chunks[i] = torch.cat([
        torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
    ])
    mask_chunks[i] = torch.cat([
        torch.tensor([1]), mask_chunks[i], torch.tensor([1])
    ])

    pad_len = chunksize - input_id_chunks[i].shape[0]
    if pad_len > 0:
        input_id_chunks[i] = torch.cat([
            input_id_chunks[i], torch.Tensor([0] * pad_len)
        ])
        mask_chunks[i] = torch.cat([
            mask_chunks[i], torch.Tensor([0] * pad_len)
        ])

for chunk in input_id_chunks:
    print(chunk)

tensor([  101,  9733,  1012,  4012, 17472,  2353,  4284,  3463,  5862,  1517,
         1006,  2449,  7318,  1007,  2255,  2656,  1010, 16798,  2509,  1517,
         9733,  1012,  4012,  1010,  4297,  1012,  1006, 17235,  2850,  4160,
         1024,  2572,  2480,  2078,  1007,  2651,  2623,  3361,  3463,  2005,
         2049,  2353,  4284,  3092,  2244,  2382,  1010, 16798,  2509,  1012,
         1528,  5658,  4341,  3445,  2410,  1003,  2000,  1002, 16065,  1012,
         1015,  4551,  1999,  1996,  2353,  4284,  1010,  4102,  2007,  1002,
        13029,  1012,  1015,  4551,  1999,  2353,  4284, 16798,  2475,  1012,
        13343,  1996,  1002,  1015,  1012,  1018,  4551, 11119,  4254,  2013,
         2095,  1011,  2058,  1011,  2095,  3431,  1999,  3097,  3863,  6165,
         2802,  1996,  4284,  1010,  5658,  4341,  3445,  2340,  1003,  4102,
         2007,  2353,  4284, 16798,  2475,  1012,  1528,  2167,  2637,  6903,
         4341,  3445,  2340,  1003,  2095,  1011,  2058,  1011, 

Here we can rejoining the splited tokens

In [ ]:
for tensor in range(len(input_id_chunks)):
  ans = answer_question(user_question1, ' '.join(tokenizer.convert_ids_to_tokens(input_id_chunks[tensor])))
  print(ans)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Query has 512 tokens.



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


('segment sales increased 12 % year - over - year to $ 23 . 1 billion . • operating income increased to $ 11 . 2 billion in the third quarter , compared with $ 2 . 5 billion in third quarter 202 # # 2 . • north america segment operating income was $ 4 . 3 billion , compared with an operating loss of $ 0 . 4 billion in third quarter 202 # # 2 . • international segment operating loss was $ 0 . 1 billion , compared with an operating loss of $ 2 . 5 billion in third quarter 202 # # 2 . • aw # # s segment operating income was $ 7 . 0 billion , compared with operating income of $ 5 . 4 billion in third quarter 202 # # 2 . • net income increased to $ 9 . 9 billion in the third quarter , or $ 0 . 94 per dil # # ut ##ed share , compared with $ 2 . 9 billion , or $ 0 . 28 per dil # # ut ##ed share , in third quarter 202 # # 2 . • third quarter 202 # # 3 net income includes a pre - tax valuation gain of $ 1 . 2 billion included in non - operating income ( expense ) from the common stock investmen

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

('[CLS]', 0.0)
Query has 512 tokens.



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


('[SEP]', 511.0)
Query has 512 tokens.



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


('[CLS]', 0.0)
Query has 512 tokens.

('.', 42.0)


In [ ]:
def expand_split_sentences(pdf_txt):
  nltk.download('punkt')
  new_chunks = nltk.sent_tokenize(pdf_txt)
  length = len(new_chunks)
  new_df = [];
  for i in range(length):
    paragraph = ""
    for j in range(i, length):
      #tmp_str = paragraph + new_chunks[j]
      tmp_token = tokenizer.encode(paragraph + new_chunks[j])
      length_token = len(tmp_token)
      if length_token < 510:
        #print(length_token)
        paragraph = paragraph + new_chunks[j]
      else:
        #print(length_token)
        break;
    #print(len(tokenizer.encode(paragraph)))
    new_df.append(paragraph)
  return new_df


Lets call the function expand_split_sentences and answer_question_F1_Score with new set of question answer pair with context


In [ ]:
user_question2 = "What were Amazon's net sales in the first quarter of 2023?"
ground_truth ="Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in the first quarter 2022."
pdf_text = "SEATTLE—(BUSINESS WIRE) April 27, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results for its first quarter ended March 31, 2023. • Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in first quarter 2022. Excluding the $2.4 billion unfavorable impact from year-over-year changes in foreign exchange rates throughout the . . . The company debuted additional Amazon Original films and series, including action comedy Shotgun Wedding, starring Jennifer Lopez and Josh Duhamel; romantic comedy Somebody I Used to Know, directed by Dave Franco; comedic thriller The Consultant, starring Christoph Waltz; and Swarm, from co-creator and executive producer Donald Glover."

In [ ]:
new_df = expand_split_sentences(pdf_text)
for new_context in new_df:
  #new_paragrapgh = new_paragrapgh + answer_question(question, answer_text)
  answer_question_F1_Score(user_question2, new_context,ground_truth)


The input has a total of 178 tokens.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Predicted Answer: "$ 127 . 4 billion"
Ground Truth Answer: "Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in the first quarter 2022."
F1 Score: 1.0
The input has a total of 136 tokens.
Predicted Answer: "$ 127 . 4 billion"
Ground Truth Answer: "Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in the first quarter 2022."
F1 Score: 1.0
The input has a total of 106 tokens.
Predicted Answer: "$ 2 . 4 billion"
Ground Truth Answer: "Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in the first quarter 2022."
F1 Score: 1.0
The input has a total of 80 tokens.
Predicted Answer: "what were amazon ' s net sales in the first quarter of 2023 ? [SEP] . . the company debuted additional amazon original films"
Ground Truth Answer: "Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in the first quarter 2022."
F1 Score: 1.0
The input

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/question_answer_dataset_financial.csv")

In [ ]:
df

,Question,Context,GroundTruth
0,What were Amazon's net sales in the first quar...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",Net sales increased 9% to $127.4 billion in th...
1,How much did net sales increase compared to th...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",Excluding the $2.4 billion unfavorable impact ...
2,What was the impact of foreign exchange rates ...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",Excluding the $2.4 billion unfavorable impact ...
3,How did North America segment sales change yea...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",North America segment sales increased 11% year...
4,What was the percentage increase in AWS segmen...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",AWS segment sales increased 16% year-over-year...
5,What was the operating income in the first qua...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",Operating income increased to $4.8 billion in ...
6,How did North America segment operating income...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",North America segment operating income was $0....
7,What was the operating income for AWS segment?,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...","AWS segment operating income was $5.1 billion,..."
8,What was Amazon's net income in the first quar...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",Net income was $3.2 billion in the first quart...
9,How did the operating cash flow change for the...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",Operating cash flow increased 38% to $54.3 bil...


Evaluate F1 score for curated financial dataset

In [ ]:
def answer_question3(question, answer_text, ground_truth):
    # Tokenize and preprocess the ground truth
    ground_truth_processed = re.sub(r' ##', '', ground_truth)
    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
    print('The input has a total of {:} tokens.'.format(len(input_ids)))
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    # Construct the list of 0s and 1s.
    segment_ids = [0] * num_seg_a + [1] * num_seg_b
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)
    outputs = model(input_ids=torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    predicted_answer = tokens[start_index]
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_index + 1, end_index + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            predicted_answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            predicted_answer += ' ' + tokens[i]
    # Tokenize and preprocess the predicted answer
    predicted_answer_processed = re.sub(r' ##', '', predicted_answer)
    # Ensure that ground truth and predicted answer have the same length
    ground_truth_processed = ground_truth_processed[:len(predicted_answer_processed)]
    # Calculate F1 score for ground truth and predicted answer
    f1_ground_truth_predicted = f1_score(list(ground_truth_processed), list(predicted_answer_processed), average='weighted')
    return predicted_answer, f1_ground_truth_predicted

In [ ]:
# Loop through rows and call the function
for index, row in df.iterrows():
    user_question = row['Question']
    context = row['Context']
    ground_truth = row['GroundTruth']
    # Call your answer_question function
    ans, f1 = answer_question3(user_question,context,ground_truth)
    # Print the result
    print(f"Question: {user_question}")
    print(f"Context: {context}")
    print(f"Answer: {ans}")
    print(f"F1 Score: {f1}")
    print("=" * 50)

The input has a total of 178 tokens.
Question: What were Amazon's net sales in the first quarter of 2023?
Context: 
SEATTLE—(BUSINESS WIRE) April 27, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results
for its first quarter ended March 31, 2023.
• Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in first quarter 2022.
Excluding the $2.4 billion unfavorable impact from year-over-year changes in foreign exchange rates throughout the
 .
 .
 .
The company debuted additional Amazon Original films and series, including action comedy Shotgun
Wedding, starring Jennifer Lopez and Josh Duhamel; romantic comedy Somebody I Used to Know, directed by Dave
Franco; comedic thriller The Consultant, starring Christoph Waltz; and Swarm, from co-creator and executive
producer Donald Glover.

Answer: $ 127 . 4 billion
F1 Score: 0.0392156862745098
The input has a total of 178 tokens.
Question: How much did net sales increase compared to the first